<a href="https://colab.research.google.com/github/ppthaw2024/Kristal-Transaction-Monitoring/blob/main/CRS_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd

# Read raw data from uploaded Excel
raw_data = pd.read_excel('/content/crs_export_KAMEL.xlsx')

# Read reference KAMEL-CRS submission template to get correct columns/order
template = pd.read_excel('/content/KAMEL - CRS Submission.xlsx', sheet_name='Accounts',header=1)
required_columns = template.columns

# Mapping logic: create a dictionary to map raw data columns to submission template
column_map = {
    'application_id': 'Number',
    'client_id': 'Holder Individual',
    'controlling_client_id': 'Holder Entity',
    'first_name': 'First Name',
    'last_name': 'Last Name',
    'residential_address_1': 'Street',
    'residential_address_2': 'Street',  # could be concatenated
    'residential_city': 'City',
    'country_of_residence': 'Country',
    'residential_postal_code': 'Postal Code',
    'primary_tax_id': 'TIN 1',
    'primary_tax_country': 'Tax Residency 1',
    'date_of_birth': 'Date of Birth',
    'booking_center': 'Type',
     'client_type': 'Number Type',
    'application_aum': 'Currency',
    'cash_balance': 'Balance'
}

# Transform and create new DataFrame for submission
submission_df = pd.DataFrame()

for kamel_col in required_columns:
    raw_col = [k for k, v in column_map.items() if v == kamel_col]

    if 'Currency' in kamel_col:
        submission_df[kamel_col] = 'USD'
    elif raw_col:
        submission_df[kamel_col] = raw_data[raw_col[0]]
    else:
        submission_df[kamel_col] = ''  # fill missing columns as empty or with logic as necessary

for index, row in submission_df.iterrows():
    if row['Number'].startswith('J'):
        submission_df.at[index, 'Is Joint?'] = 'T'
    else:
        submission_df.at[index, 'Is Joint?'] = 'F'

    ts = pd.Timestamp(row['Date of Birth'])
    row['Date of Birth'] = ts.strftime('%d/%m/%Y')
  #  print(str(row['Date of Birth']))



# Save to file for download or further processing
submission_df.to_excel('submission_ready.xlsx', index=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')